In [ ]:
import pandas as pd

In [ ]:
import plotly.express as px

In [ ]:
import plotly.io as pio

In [ ]:
import plotly.graph_objects as go

In [ ]:
pio.templates.default = "plotly_white"

In [ ]:
data=pd.read_csv("C:/Users/Francis Mwangi/Downloads/rfm_data.csv")

In [ ]:
print(data.head())

In [ ]:
from datetime import datetime


In [ ]:
# Convert 'PurchaseDate' to datetime

data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])



In [ ]:
# Confirming data in PurchaseDate
print(data['PurchaseDate'].head())


In [ ]:
current_date = datetime.now().date()

In [ ]:
print(current_date)

In [ ]:
# Calculate Recency
data['Recency'] = (current_date - data['PurchaseDate'].dt.date.days)




In [ ]:
# Calculate Frequency

frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()

In [ ]:
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)

In [ ]:
data = data.merge(frequency_data, on='CustomerID', how='left')

In [ ]:
# Calculate Monetary Value

monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()

In [ ]:
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)

In [ ]:
data = data.merge(monetary_data, on='CustomerID', how='left')

In [ ]:
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()

In [ ]:
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)


In [ ]:
data = data.merge(monetary_data, on='CustomerID', how='left')

In [ ]:
print(data.head())

In [ ]:
# Define scoring criteria for each RFM value

recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)

In [ ]:
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency

In [ ]:
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

In [ ]:
# Calculate RFM scores

data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)

In [ ]:
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

In [ ]:
# Convert RFM scores to numeric type

data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

In [ ]:
# Calculate RFM score by combining the individual scores

data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']


In [ ]:
# Create RFM segments based on the RFM score

segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)

In [ ]:
segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

In [ ]:
pastel_colors = px.colors.qualitative.Pastel


In [ ]:
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count', 
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')


In [ ]:
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

In [ ]:
fig_segment_dist.show()

In [ ]:
data['RFM Customer Segments'] = ''


In [ ]:
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"


In [ ]:
print(data[['CustomerID', 'RFM Customer Segments']])


In [ ]:
segment_product_counts = data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts, 
                                         path=['Value Segment', 'RFM Customer Segments'], 
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

In [ ]:
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()

In [ ]:
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

In [ ]:
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [ ]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = data['RFM Customer Segments'].value_counts()

In [ ]:
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

In [ ]:
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)


In [ ]:
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [ ]:
segment_scores = data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()


In [ ]:
fig = go.Figure()


In [ ]:
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

In [ ]:
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

In [ ]:
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))


In [ ]:
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)
fig.show()